In [1]:
import warnings
from typing import Sequence, Tuple, Union

import torch
import torch.nn as nn

from monai.networks.blocks.convolutions import Convolution, ResidualUnit
from monai.networks.layers.factories import Act, Norm
from monai.networks.layers.simplelayers import SkipConnection
from monai.utils import alias, export

import matplotlib.pyplot as plt
import pandas as pd
from monai.losses import DiceLoss
from monai.metrics import DiceMetric, compute_meandice
import torch
import torch.nn as nn
import numpy as np
import os
import torch.optim as opt
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import cv2
import h5py
from scipy.ndimage.interpolation import zoom
import torchvision.transforms as T
import random
from scipy.ndimage.interpolation import zoom
from torch_geometric.nn import SAGEConv
from torch.cuda.amp import GradScaler, autocast
from monai.networks.nets import UNet
from Graph_GNN import UNet_GNN_pass, UNet_GNN_linear

In [2]:
start = []
end = []
for i in range(625):
    if i >= 0 and i <= 24:
        if i == 0:
            start.append(0)
            end.append(1)
            start.append(0)
            end.append(50)
        elif i == 24:
            start.append(24)
            end.append(23)
            start.append(24)
            end.append(49)
        else:
            start.append(i)
            end.append(i-1)
            start.append(i)
            end.append(i+1)
            start.append(i)
            end.append(i+25)
    elif (i >= 600 and i <= 624):
        if i == 600:
            start.append(600)
            end.append(575)
            start.append(600)
            end.append(601)
        elif i == 624:
            start.append(600)
            end.append(575)
            start.append(600)
            end.append(601)
        else:
            start.append(i)
            end.append(i-1)
            start.append(i)
            end.append(i+1)
            start.append(i)
            end.append(i-25)
    elif i % 25 == 0:
        start.append(i)
        end.append(i+1)
        start.append(i)
        end.append(i+25)
        start.append(i)
        end.append(i-25)
    elif i % 25 == 24:
        start.append(i)
        end.append(i-1)
        start.append(i)
        end.append(i+25)
        start.append(i)
        end.append(i-25)     
    else:
        start.append(i)
        end.append(i-1)
        start.append(i)
        end.append(i+25)
        start.append(i)
        end.append(i-25)  
        start.append(i)
        end.append(i+1)
edges = torch.tensor([start, end], dtype=torch.long)
    
def one_hot(labels: torch.Tensor, num_classes: int, dtype: torch.dtype = torch.float, dim: int = 1) -> torch.Tensor:
    # if `dim` is bigger, add singleton dim at the end
    if labels.ndim < dim + 1:
        shape = list(labels.shape) + [1] * (dim + 1 - len(labels.shape))
        labels = torch.reshape(labels, shape)

    sh = list(labels.shape)

    if sh[dim] != 1:
        raise AssertionError("labels should have a channel with length equal to one.")

    sh[dim] = num_classes

    o = torch.zeros(size=sh, dtype=dtype, device=labels.device)
    labels = o.scatter_(dim=dim, index=labels.long(), value=1)

    return labels

In [3]:
def non_zero_acc(pre, real):
    real = real.flatten()
    pre = pre.flatten()
    non_zero_real = real[((real == 1) | (pre == 1))]
    non_zero_pre = pre[((real == 1) | (pre == 1))]
    return np.sum(non_zero_real == non_zero_pre)/ len(non_zero_pre)
def np_dice(pre, real):
    k=1
    before_dice = np.sum(pre[real==k]==k)*2.0 / (np.sum(pre[pre==k]==k) + np.sum(real[real==k]==k))
    k=0
    back_dice = np.sum(pre[real==k]==k)*2.0 / (np.sum(pre[pre==k]==k) + np.sum(real[real==k]==k))
    return before_dice, back_dice

In [4]:
base = './'
data_path = [base + 'val/' + x for x in os.listdir(base + 'val/')]
names = [x.split('/')[-1].split('_')[0] for x in data_path]
counts = pd.Series(names).value_counts().to_dict()
templete =  './val/{x}_{y}.h5'

In [8]:
def test_one_case(name, counts, model, device, unet=False):
    count = counts[name]
    real = []
    out = []
    for i in range(1, count-2):
        paths = []
        for j in range(i-1, i+2):
            paths.append(templete.format(x=name, y=j))
        xs = []
        f = h5py.File(paths[1], 'r')
        y = f['gt'][:]
        real.append(y)
        f.close()
        for path in paths:
            f = h5py.File(path, 'r')
            x = f['ct'][:].astype(np.float)
            x = torch.from_numpy(x)
            x = x.float().view(1, 400, 400)
            xs.append(x)  
            f.close()
        x = torch.cat(xs, dim=0).view(1, 3, 400, 400)
        x = x.to(device)
        if unet:
            outputs = model(x)
        else:
            outputs = model(x, device, edges)
        outputs = torch.nn.functional.softmax(outputs, 1)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.view(400, 400).detach().cpu().numpy()
        out.append(predicted)
        
    out = np.array(out)
    real = np.array(real)
    return np.mean(out == real), non_zero_acc(out, real), np_dice(out, real)

In [6]:
def test_one_model(model, device, unet):
    ans = pd.DataFrame()
    model.to(device)
    model.eval()
    names = []
    means = []
    non_means = []
    dice_backs = []
    dice_befores = []
    for name in tqdm(counts.keys()):
        res = test_one_case(name, counts, model, device, unet=unet)
        names.append(name)
        means.append(res[0])
        non_means.append(res[1])
        dice_befores.append(res[2][0])
        dice_backs.append(res[2][1])
    ans['Name'] = names
    ans['Mean'] = means
    ans['Mean without backgroud'] = non_means
    ans['Dice Coefficient (label 1)'] = dice_befores
    ans['Dice Coefficient (label 0)'] = dice_backs
    return ans

In [29]:
model_unet = UNet(
    dimensions=2,
    in_channels=3,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
model_unet.load_state_dict(torch.load('./model_weights/best_unet_weight.pt'))
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
unet_stat = test_one_model(model_unet, device, True)
del model_unet

In [30]:
unet_stat

,Name,Mean,Mean without backgroud,Dice Coefficient (label 1),Dice Coefficient (label 0)
0,1178184813,0.977016,0.582942,0.736530,0.987984
1,1112425214,0.979377,0.622806,0.767567,0.989210
2,1103364672,0.980481,0.633381,0.775546,0.989797
3,1112441303,0.976733,0.622461,0.767305,0.987754
4,1132400990,0.975723,0.610724,0.758322,0.987220
5,1105185175,0.974944,0.604099,0.753194,0.986802
6,1082205524,0.976903,0.600646,0.750504,0.987891
7,1126357612,0.862086,0.026253,0.051162,0.925639
8,1132388375,0.974717,0.596470,0.747236,0.986693


In [ ]:
model_gnn_pass = UNet_GNN_pass(
    dimensions=2,
    in_channels=3,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
model_gnn_pass.load_state_dict(torch.load('./model_weights/best_sage_conv_weight.pt'))
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
gnn_pass_stat = test_one_model(model_gnn_pass, device, False)
gnn_pass_stat.to_csv('gnn_pass_stat.csv')
del model_gnn_pass

In [ ]:
model_gnn_linear = UNet_GNN_linear(
    dimensions=2,
    in_channels=3,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
model_gnn_linear.load_state_dict(torch.load('./model_weights/best_sage_conv_linear_weight.pt'))
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
gnn_linear_stat = get_stat(model_gnn_linear, device)
del model_gnn_linear